In [ ]:
#| default_exp questionnaires_handler


In [ ]:
#| hide
from nbdev.showdoc import *

ModuleNotFoundError: No module named 'nbdev'

In [ ]:
#| export
import pandas as pd
import numpy as np

In [ ]:

#| export

# Here is a function that can convert a column to string, removing '.0' from floats
import numpy as np
import warnings

def convert_to_string(x):
    return str(int(x)) if isinstance(x, float) and x.is_integer() else str(x)

def normalize_answers(orig_answer: pd.Series) -> pd.Series:
    """
    Normalize the answers from string.

    Args:
        orig_answer (pd.Series): The original answer series.

    Returns:
        pd.Series: The normalized answer series.
    """
    # Convert the entire series to strings, np.nan will become 'nan'
    normalized_answer = orig_answer.astype(str)

    # Replace float-like strings with integer-like strings, ignoring 'nan'
    normalized_answer = normalized_answer.str.replace(r"\.0$", "", regex=True)
    normalized_answer = normalized_answer.replace("nan", np.nan)

    return normalized_answer

def check_invalid_values(code_df, code_from, normalized_answer):
    """
    Check if values in normalized_answer exist in code_df[code_from], excluding np.nan.

    Args:
        code_df (pd.DataFrame): The DataFrame containing the code mappings.
        code_from (str): The column name to check in code_df.
        normalized_answer (pd.Series): The normalized answer series.

    Returns:
        None: Prints the invalid values found, if any.
    """
    # Check if values in normalized_answer exist in code_df[code_from], excluding np.nan
    valid_values = set(code_df[code_from].astype(str))
    answer_values = set(normalized_answer.dropna())
    invalid_values = answer_values - valid_values

    if invalid_values:
        warnings.warn(f"Invalid values found: {invalid_values}")

def transform_answers(
    tab_field_name: str,
    orig_answer: pd.Series,
    transform_from: str,
    transform_to: str,
    dict_df: pd.DataFrame,
    mapping_df: pd.DataFrame,
) -> pd.Series:
    code_from = transform_from.lower()
    code_to = transform_to.lower()
    assert code_from in ["hebrew", "english", "coding"]
    assert code_to in ["hebrew", "english", "coding"]

    # the index of the dict_df is the tabular_field_name
    if isinstance(dict_df.loc[tab_field_name]["data_coding"], pd.Series):
        code_string = convert_to_string(dict_df.loc[tab_field_name]["data_coding"].iloc[0])
    else:
        code_string = convert_to_string(dict_df.loc[tab_field_name]["data_coding"])

    code_df = mapping_df[mapping_df["code_number"] == code_string].copy()
    #Make sure no leading 0s for coding values
    code_df["coding"] = code_df["coding"].astype(int).astype(str)

    coding = dict(zip(code_df[code_from].astype(str), code_df[code_to]))

    normalized_answer = normalize_answers(orig_answer)

    check_invalid_values(code_df, code_from, normalized_answer)
    # Replace only valid values, preserving np.nan
    transformed_answer = normalized_answer.replace(coding)

    #transform to categorical varaible
    transformed_answer = transformed_answer.astype("category")


    return transformed_answer



def transform_dataframe(
    df: pd.DataFrame,
    transform_from: str,
    transform_to: str,
    dict_df: pd.DataFrame,
    mapping_df: pd.DataFrame,
) -> pd.DataFrame:
    transformed_df = df.copy()
    for column in transformed_df.columns:
        if column in dict_df.index:
            data_coding = dict_df.loc[column, 'data_coding']
            # Handle the case where data_coding is a Series (multiple entries)
            if isinstance(data_coding, pd.Series):
                # Proceed only if all data_codings are consistent
                if data_coding.nunique() == 1 and pd.notna(data_coding.iloc[0]):
                    transformed_df[column] = transform_answers(
                        column,
                        transformed_df[column],
                        transform_from,
                        transform_to,
                        dict_df,
                        mapping_df
                    )
            else:  # Single value for data_coding
                if pd.notna(data_coding):
                    transformed_df[column] = transform_answers(
                        column,
                        transformed_df[column],
                        transform_from,
                        transform_to,
                        dict_df,
                        mapping_df
                    )
    return transformed_df




NameError: name 'pd' is not defined

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()